
# E1 - Model Deployment in Used Vehicle Price Prediction

## Introduction

- 1.2 Million listings scraped from TrueCar.com - Price, Mileage, Make, Model dataset from Kaggle: [data](https://www.kaggle.com/jpayne/852k-used-car-listings)
- Each observation represents the price of an used car

In [2]:
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [3]:
data = pd.read_csv('https://github.com/sergiomora03/AdvancedTopicsAnalytics/raw/main/datasets/dataTrain_carListings.zip')

In [4]:
data.head()

,Price,Year,Mileage,State,Make,Model
0,21490,2014,31909,MD,Nissan,MuranoAWD
1,21250,2016,25741,KY,Chevrolet,CamaroCoupe
2,20925,2016,24633,SC,Hyundai,Santa
3,14500,2012,84026,OK,Jeep,Grand
4,32488,2013,22816,TN,Jeep,Wrangler


In [5]:
data.shape

(500000, 6)

In [6]:
data.columns

Index(['Price', 'Year', 'Mileage', 'State', 'Make', 'Model'], dtype='object')

# Exercise P0.1 (50%)

Develop a machine learning model that predicts the price of the of car using as an input ['Year', 'Mileage', 'State', 'Make', 'Model']

#### Evaluation:
- 25% - Performance of the models using a manually implemented K-Fold (K=10) cross-validation
- 25% - Notebook explaining the process for selecting the best model. You must specify how the calibration of each of the parameters is done and how these change the performance of the model. It is expected that a clear comparison will be made of all implemented models.. Present the most relevant conslusions about the whole process.


Feature engineering

In [7]:
State_dummies = pd.get_dummies(data.State, prefix = 'State', dtype= int)
data = pd.concat([data, State_dummies], axis=1)

In [8]:
Make_dummies = pd.get_dummies(data.Make, prefix = 'Make', dtype= int)
data = pd.concat([data, Make_dummies], axis=1)

In [9]:
Model_dummies = pd.get_dummies(data.Model, prefix = 'Model', dtype= int)
data = pd.concat([data, Model_dummies], axis=1)

In [6]:
#Aplicar Label encoder a la variable model
from sklearn.preprocessing import LabelEncoder


In [10]:
categorical_features = ['State', 'Make', 'Model']

In [8]:
data[categorical_features] = data[categorical_features].apply(LabelEncoder().fit_transform)
data

,Price,Year,Mileage,State,Make,Model
0,21490,2014,31909,20,27,305
1,21250,2016,25741,17,6,83
2,20925,2016,24633,40,14,400
3,14500,2012,84026,36,17,248
4,32488,2013,22816,42,17,489
...,...,...,...,...,...,...
499995,16997,2013,40393,43,2,0
499996,11990,2012,91794,47,14,400
499997,49991,2014,39107,25,10,438
499998,21888,2010,62556,30,8,378


In [11]:
#drop initial categorical features
data.drop(columns = categorical_features, inplace = True)

In [12]:
data.head()

,Price,Year,Mileage,State_ AK,State_ AL,State_ AR,State_ AZ,State_ CA,State_ CO,State_ CT,...,Model_Yaris4dr,Model_YarisBase,Model_YarisLE,Model_Yukon,Model_Yukon2WD,Model_Yukon4WD,Model_Yukon4dr,Model_tC2dr,Model_xB5dr,Model_xD5dr
0,21490,2014,31909,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21250,2016,25741,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20925,2016,24633,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14500,2012,84026,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32488,2013,22816,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
data[:1]

,Price,Year,Mileage,State_ AK,State_ AL,State_ AR,State_ AZ,State_ CA,State_ CO,State_ CT,...,Model_Yaris4dr,Model_YarisBase,Model_YarisLE,Model_Yukon,Model_Yukon2WD,Model_Yukon4WD,Model_Yukon4dr,Model_tC2dr,Model_xB5dr,Model_xD5dr
0,21490,2014,31909,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
features = ['Year', 'Mileage'] + list(State_dummies.columns) + list(Make_dummies.columns) + list(Model_dummies.columns)
features

['Year',
 'Mileage',
 'State_ AK',
 'State_ AL',
 'State_ AR',
 'State_ AZ',
 'State_ CA',
 'State_ CO',
 'State_ CT',
 'State_ DC',
 'State_ DE',
 'State_ FL',
 'State_ GA',
 'State_ HI',
 'State_ IA',
 'State_ ID',
 'State_ IL',
 'State_ IN',
 'State_ KS',
 'State_ KY',
 'State_ LA',
 'State_ MA',
 'State_ MD',
 'State_ ME',
 'State_ MI',
 'State_ MN',
 'State_ MO',
 'State_ MS',
 'State_ MT',
 'State_ NC',
 'State_ ND',
 'State_ NE',
 'State_ NH',
 'State_ NJ',
 'State_ NM',
 'State_ NV',
 'State_ NY',
 'State_ OH',
 'State_ OK',
 'State_ OR',
 'State_ PA',
 'State_ RI',
 'State_ SC',
 'State_ SD',
 'State_ TN',
 'State_ TX',
 'State_ UT',
 'State_ VA',
 'State_ VT',
 'State_ WA',
 'State_ WI',
 'State_ WV',
 'State_ WY',
 'Make_Acura',
 'Make_Audi',
 'Make_BMW',
 'Make_Bentley',
 'Make_Buick',
 'Make_Cadillac',
 'Make_Chevrolet',
 'Make_Chrysler',
 'Make_Dodge',
 'Make_FIAT',
 'Make_Ford',
 'Make_Freightliner',
 'Make_GMC',
 'Make_Honda',
 'Make_Hyundai',
 'Make_INFINITI',
 'Make_J

In [9]:
features = list(data.columns)
features.remove('Price')
features

['Year', 'Mileage', 'State', 'Make', 'Model']

Test Train Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_data, test_data, train_y, test_y = train_test_split(data[features], data.Price,
                                                   random_state = 1603, test_size = 0.25)

Modeling

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
rf = RandomForestClassifier(n_estimators=100, n_jobs = -1, random_state = 11, max_depth=3)

In [18]:
rf.fit(train_data, train_y)

RandomForestClassifier(max_depth=3, n_jobs=-1, random_state=11)

In [110]:
test_data[:1]

,Year,Mileage,State_ AK,State_ AL,State_ AR,State_ AZ,State_ CA,State_ CO,State_ CT,State_ DC,...,Model_Yaris4dr,Model_YarisBase,Model_YarisLE,Model_Yukon,Model_Yukon2WD,Model_Yukon4WD,Model_Yukon4dr,Model_tC2dr,Model_xB5dr,Model_xD5dr
207873,2015,35502,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
test_data[:1].shape

(1, 616)

In [99]:
v = rf.predict(test_data[:1])[0]
v

14995

In [19]:
import joblib

In [21]:
joblib.dump(rf, 'vehicle_price_prediction.pkl', compress=3)

['vehicle_price_prediction.pkl']

# Exercise P0.2 (50%)

Create an API of the model.

Example:
![](https://github.com/sergiomora03/AdvancedTopicsAnalytics/blob/main/notebooks/img/img015.PNG?raw=true)

#### Evaluation:
- 40% - API hosted on a cloud service
- 10% - Show screenshots of the model doing the predictions on the local machine


In [22]:
!pip install flask-restx

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.8 MB 2.2 MB/s eta 0:00:02
   --------- ------------------------------ 0.7/2.8 MB 8.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 19.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 19.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 19.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 19.4 MB/s eta 0:00:01
   ---------------------------------------  2.8/2.8 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/52.8 kB ? eta -:--:--
   ---------------------------------------- 52.8/52.8 kB 2.7 MB/s eta 0:00:00


In [149]:
from flask import Flask
from flask_restx import Api, Resource, fields
import joblib

In [173]:
app = Flask(__name__)

api = Api(
    app,
    version='1.0',
    title='Vehicle Price Prediction API',
    description='Vehicle Price Prediction API')

ns = api.namespace('predict',
     description='Vehicle Price Prediction')

parser = api.parser()

parser.add_argument(
    'Year',
    type=int,
    required=True,
    help='Year of vehicle to predict price',
    location='args')

parser.add_argument(
    'Mileage',
    type=int,
    required=True,
    help='Mileage of vehicle to predict price',
    location='args')

parser.add_argument(
    'State',
    type=str,
    required=True,
    help='State of vehicle to predict price',
    location='args')

parser.add_argument(
    'Make',
    type=str,
    required=True,
    help='Make of vehicle to predict price',
    location='args')

parser.add_argument(
    'Model',
    type=str,
    required=True,
    help='Model of vehicle to predict price',
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [192]:
from m06_model_deployment import predict_proba

In [193]:
@ns.route('/')
class VehPricePredictionApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()

        Year = args['Year']
        Mileage = args['Mileage']
        State = args['State']
        Make = args['Make']
        Model = args['Model']

        return {
        "result": predict_proba(Year, Mileage, State, Make, Model)
        }, 200

In [194]:
#Run API

app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.13:5001
Press CTRL+C to quit
127.0.0.1 - - [27/Aug/2024 21:10:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2024 21:10:44] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 21:10:44] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 21:10:44] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 21:10:44] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Aug/2024 21:10:45] "GET /swagger.json HTTP/1.1" 200 -
c:\Users\Acer\OneDrive\Documentos\MAIN\Tópicos Avanzados de Analítica\model_deployment\m06_model_deployment.py:654: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfram

IMAGENES

<img src="https://raw.githubusercontent.com/linamvaron/AdvancedTopicsAnalytics/main/Images/E1_GET.png" alt="API" width="1000"/>

<img src="https://raw.githubusercontent.com/linamvaron/AdvancedTopicsAnalytics/main/Images/E1_EXECUTE.png" alt="API" width="1000"/>